In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn import discriminant_analysis, linear_model

In [2]:
data = pd.read_csv("wine-reviews/winemag-data-130k-v2.csv")

In [3]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## A quick function to extract the year from the title

In [4]:
def get_year(string):
    '''
    find the year of the wine based on the title
    '''
    ## some of the titles say 'Years Old' which doesn't tell us the actual year
    nums = "".join([s for s in string if s.isdigit() and not 'Years' in string])
    if len(nums) != 4:
            nums = np.nan
    return nums

In [5]:
data['year'] = data['title'].map(get_year)

In [6]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


Let's look to see if there is a correlation between price and score

In [7]:
corr = data['points'].corr(data['price'])
print(corr)

0.4161667418606222


Later I will go on to do some regression to see if we can predict the points based on the wine. For now, let's move it to a simple table to do exploratory analysis

In [8]:
data.to_csv("output1.csv")

## Let's break this up so we can start building a predictor

In [9]:
analytics = data[['year', 'country', 'price', 'province', 'region_1', 'taster_name', 'variety', 'points', 'winery']]

In [10]:
analytics.reset_index()

,index,year,country,price,province,region_1,taster_name,variety,points,winery
0,0,2013,Italy,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,White Blend,87,Nicosia
1,1,2011,Portugal,15.0,Douro,NaN,Roger Voss,Portuguese Red,87,Quinta dos Avidagos
2,2,2013,US,14.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,87,Rainstorm
3,3,2013,US,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,Riesling,87,St. Julian
4,4,2012,US,65.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Noir,87,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...
129966,129966,2013,Germany,28.0,Mosel,NaN,Anna Lee C. Iijima,Riesling,90,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,2004,US,75.0,Oregon,Oregon,Paul Gregutt,Pinot Noir,90,Citation
129968,129968,2013,France,30.0,Alsace,Alsace,Roger Voss,Gewürztraminer,90,Domaine Gresser
129969,129969,2012,France,32.0,Alsace,Alsace,Roger Voss,Pinot Gris,90,Domaine Marcel Deiss


First, let's drop some nulls to get the best results

In [11]:
analytics.dropna(axis=0)

,year,country,price,province,region_1,taster_name,variety,points,winery
2,2013,US,14.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,87,Rainstorm
3,2013,US,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,Riesling,87,St. Julian
4,2012,US,65.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Noir,87,Sweet Cheeks
5,2011,Spain,15.0,Northern Spain,Navarra,Michael Schachner,Tempranillo-Merlot,87,Tandem
6,2013,Italy,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Frappato,87,Terre di Giurfo
...,...,...,...,...,...,...,...,...,...
129965,2013,France,28.0,Alsace,Alsace,Roger Voss,Pinot Gris,90,Domaine Rieflé-Landmann
129967,2004,US,75.0,Oregon,Oregon,Paul Gregutt,Pinot Noir,90,Citation
129968,2013,France,30.0,Alsace,Alsace,Roger Voss,Gewürztraminer,90,Domaine Gresser
129969,2012,France,32.0,Alsace,Alsace,Roger Voss,Pinot Gris,90,Domaine Marcel Deiss


For the categorical variables, we will need some dummy variables to represent them. Let's see how many levels we're dealing with

In [12]:
print(f'There are {len(set(analytics.country))} countries')
print(f'There are {len(set(analytics.province))} provinces')
print(f'There are {len(set(analytics.region_1))} regions')
print(f'There are {len(set(analytics.taster_name))} tasters')
print(f'There are {len(set(analytics.variety))} varieties')
print(f'There are {len(analytics.index)} rows that have after dropping NAs')

There are 44 countries
There are 426 provinces
There are 1230 regions
There are 20 tasters
There are 708 varieties
There are 129971 rows that have after dropping NAs


In [13]:
# I'm going to limit to our 25 most popular wines
top_kinds = pd.value_counts(analytics.variety)[0:25]


In [14]:
analytics = analytics[analytics['variety'].isin(list(top_kinds.index))]


In [15]:
# create the dummy variables
varieties = pd.get_dummies(analytics.variety)
var_cols = [col for col in varieties.columns]
countries = pd.get_dummies(analytics.country)
country_cols = [col for col in countries.columns]
tasters = pd.get_dummies(analytics.taster_name)
taster_cols = [col for col in tasters.columns]

In [16]:
# merge data by index
cleaned = varieties.merge(countries, how='outer', left_index=True, right_index=True)
cleaned = cleaned.merge(tasters, how='outer', left_index=True, right_index=True)
cleaned = cleaned.merge(analytics[['price', 'year', 'points']], how='outer', left_index=True, right_index=True)
cleaned.dropna()
cleaned.reset_index()
cleaned.head()

,Bordeaux-style Red Blend,Bordeaux-style White Blend,Cabernet Franc,Cabernet Sauvignon,Champagne Blend,Chardonnay,Grüner Veltliner,Malbec,Merlot,Nebbiolo,...,Michael Schachner,Mike DeSimone,Paul Gregutt,Roger Voss,Sean P. Sullivan,Susan Kostrzewa,Virginie Boone,price,year,points
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,2013,87
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,15.0,2011,87
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,14.0,2013,87
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13.0,2013,87
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,65.0,2012,87


In [17]:
price_data = cleaned.dropna()

In [18]:
print(price_data.columns.values)

['Bordeaux-style Red Blend' 'Bordeaux-style White Blend' 'Cabernet Franc'
 'Cabernet Sauvignon' 'Champagne Blend' 'Chardonnay' 'Grüner Veltliner'
 'Malbec' 'Merlot' 'Nebbiolo' 'Pinot Gris' 'Pinot Noir' 'Portuguese Red'
 'Portuguese White' 'Red Blend' 'Rhône-style Red Blend' 'Riesling' 'Rosé'
 'Sangiovese' 'Sauvignon Blanc' 'Sparkling Blend' 'Syrah' 'Tempranillo'
 'White Blend' 'Zinfandel' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Brazil' 'Bulgaria' 'Canada' 'Chile' 'Croatia' 'Cyprus' 'Czech Republic'
 'England' 'France' 'Georgia' 'Germany' 'Greece' 'Hungary' 'India'
 'Israel' 'Italy' 'Lebanon' 'Luxembourg' 'Macedonia' 'Mexico' 'Moldova'
 'Morocco' 'New Zealand' 'Peru' 'Portugal' 'Romania' 'Serbia' 'Slovakia'
 'Slovenia' 'South Africa' 'Spain' 'Switzerland' 'Turkey' 'US' 'Ukraine'
 'Uruguay' 'Alexander Peartree' 'Anna Lee C. Iijima' 'Anne Krebiehl\xa0MW'
 'Carrie Dykes' 'Christina Pickard' 'Fiona Adams' 'Jeff Jenssen'
 'Jim Gordon' 'Joe Czerwinski' 'Kerin O’Keefe' 'Lauren Buzzeo'
 '

In [19]:
dep_vars = list(price_data.columns.values)[:-1]
print(len(dep_vars))

86


In [20]:
X = price_data[dep_vars]
y = price_data['points']

In [21]:
regr = discriminant_analysis.LinearDiscriminantAnalysis()
regr.fit(X, y)

LinearDiscriminantAnalysis()

In [22]:
wine_list = [0] * 25
ctry_list = [0] * 40
taste_list = [0] * 19

In [23]:
wine_list[19] = 1
ctry_list[23] = 1
taste_list[10] = 1
price_input = 275
year_input = 1950
inputs = wine_list + ctry_list + taste_list
inputs.append(price_input)
inputs.append(year_input)

In [24]:
print(regr.predict([inputs]))

[98]


## What if I can create a data point for every possible combination?

Loop through all the wine types, countries, tasters, then every year and bottles up to $1000?

In [25]:
## number of data points
print(24 * 40 * 19 * 20 * 20)

7296000


## We have a fitting model! Now let's see if we can get better way to change the inputs

In order to best do this, we should probably set up some server to interact with it